# Load config

In [ ]:
# Load config
from config import UNetTraining
config = UNetTraining.Configuration()

In [ ]:
if 0:
    %run "1_data_preparation.ipynb"
    %run "2_model.ipynb"

In [ ]:
import os
import time

# Define callbacks for the early stopping of training, LearningRateScheduler and model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard


checkpoint = ModelCheckpoint(config.model_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = False)

#reduceonplatea; It can be useful when using adam as optimizer
#Reduce learning rate when a metric has stopped improving (after some patience,reduce by a factor of 0.33, new_lr = lr * factor).
#cooldown: number of epochs to wait before resuming normal operation after lr has been reduced.
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.33,
                                   patience=4, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=4, min_lr=1e-16)

#early = EarlyStopping(monitor="val_loss", mode="min", verbose=2, patience=15)
timestr = time.strftime("%Y%m%d-%H%M")

log_dir = os.path.join('./logs','UNet_{}_{}_{}_{}_{}'.format(timestr,OPTIMIZER_NAME,LOSS_NAME,chs, config.input_shape[0]))
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

callbacks_list = [checkpoint, tensorboard] #reduceLROnPlat is not required with adaDelta

In [ ]:
%%time
loss_history = model.fit(datagen.flow(X_train_mini_np, y_train_mini_np,
                                batch_size=config.BATCH_SIZE, subset='training'),
                        epochs=config.NB_EPOCHS,
                        steps_per_epoch=len(X_train_mini_np) // config.BATCH_SIZE, #config.MAX_TRAIN_STEPS, # steps_per_epoch * epochs <= # data instances = 94
                        validation_data=datagen.flow(X_val_np, y_val_np, 
                                batch_size=config.BATCH_SIZE, subset='validation'),
                        validation_steps=len(X_train_mini_np) // config.BATCH_SIZE,
                        batch_size=config.BATCH_SIZE,
                        callbacks=callbacks_list
                        # use_multiprocessing=True # the generator is not very thread safe
                )
print("Training completed.")

In [ ]:
if not os.path.exists(config.model_path):
    os.makedirs(config.model_path)
acc     = round(100*loss_history.history['accuracy'][-1], 4)
val_acc = round(100*loss_history.history['val_accuracy'][-1], 4)
model_path = os.path.join(config.model_path,'UNet_{}_{}_{}_{}_{}_{}.h5'.format(timestr,OPTIMIZER_NAME,LOSS_NAME,config.input_shape[0], acc, val_acc))
print(model_path)
model.save(model_path)